# NLSQ Large Dataset Fitting Demonstration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/large_dataset_demo.ipynb)

**Requirements:** Python 3.12 or higher

This notebook demonstrates the capabilities of NLSQ for handling very large datasets with automatic memory management, chunking, and sampling strategies.

## Key Features:
- Memory estimation for datasets from 100K to 100M+ points
- Automatic chunking for datasets that don't fit in memory
- Sampling strategies for extremely large datasets
- Progress reporting for long-running fits
- Memory-aware optimization
- Advanced configuration management with context managers

## Setup and Imports

In [1]:
#!/usr/bin/env python3
"""
Demonstration of NLSQ Large Dataset Fitting Capabilities with Advanced Features
"""

# Check Python version
import sys

print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor} meets requirements")

import time

import jax.numpy as jnp
import numpy as np

from nlsq import (
    AlgorithmSelector,
    CurveFit,
    LargeDatasetConfig,
    LargeDatasetFitter,
    LDMemoryConfig,
    # New advanced features
    MemoryConfig,
    __version__,
    auto_select_algorithm,
    configure_for_large_datasets,
    curve_fit_large,
    estimate_memory_requirements,
    fit_large_dataset,
    get_memory_config,
    large_dataset_context,
    memory_context,
    set_memory_limits,
)

print(f"NLSQ version: {__version__}")
print("NLSQ Large Dataset Demo - Enhanced Version")
print("Including advanced memory management and algorithm selection")


# Define our model functions
def exponential_decay(x, a, b, c):
    """Exponential decay model with offset: y = a * exp(-b * x) + c"""
    return a * jnp.exp(-b * x) + c


def polynomial_model(x, a, b, c, d):
    """Polynomial model: y = a*x^3 + b*x^2 + c*x + d"""
    return a * x**3 + b * x**2 + c * x + d


def gaussian(x, a, mu, sigma, offset):
    """Gaussian model: y = a * exp(-((x - mu)^2) / (2*sigma^2)) + offset"""
    return a * jnp.exp(-((x - mu) ** 2) / (2 * sigma**2)) + offset


def complex_model(x, a, b, c, d, e, f):
    """Complex model with many parameters for algorithm selection testing"""
    return a * jnp.exp(-b * x) + c * jnp.sin(d * x) + e * x**2 + f

✅ Python 3.12 meets requirements


NLSQ version: 0.1.0.post66
NLSQ Large Dataset Demo - Enhanced Version
Including advanced memory management and algorithm selection


## 1. Memory Estimation Demo

First, let's understand how much memory different dataset sizes require and what processing strategies NLSQ recommends.

In [2]:
def demo_memory_estimation():
    """Demonstrate memory estimation capabilities."""
    print("=" * 60)
    print("MEMORY ESTIMATION DEMO")
    print("=" * 60)

    # Estimate requirements for different dataset sizes
    test_cases = [
        (100_000, 3, "Small dataset"),
        (1_000_000, 3, "Medium dataset"),
        (10_000_000, 3, "Large dataset"),
        (50_000_000, 3, "Very large dataset"),
        (100_000_000, 3, "Extremely large dataset"),
    ]

    for n_points, n_params, description in test_cases:
        stats = estimate_memory_requirements(n_points, n_params)

        print(f"\n{description} ({n_points:,} points, {n_params} parameters):")
        print(f"  Memory estimate: {stats.total_memory_estimate_gb:.2f} GB")
        print(f"  Chunk size: {stats.recommended_chunk_size:,}")
        print(f"  Number of chunks: {stats.n_chunks}")

        if stats.requires_sampling:
            print("  Strategy: Sampling recommended")
        elif stats.n_chunks == 1:
            print("  Strategy: Single chunk (fits in memory)")
        else:
            print("  Strategy: Chunked processing")


# Run the demo
demo_memory_estimation()

MEMORY ESTIMATION DEMO

Small dataset (100,000 points, 3 parameters):
  Memory estimate: 0.01 GB
  Chunk size: 100,000
  Number of chunks: 1
  Strategy: Single chunk (fits in memory)

Medium dataset (1,000,000 points, 3 parameters):
  Memory estimate: 0.14 GB
  Chunk size: 1,000,000
  Number of chunks: 1
  Strategy: Single chunk (fits in memory)

Large dataset (10,000,000 points, 3 parameters):
  Memory estimate: 1.36 GB
  Chunk size: 1,000,000
  Number of chunks: 10
  Strategy: Chunked processing

Very large dataset (50,000,000 points, 3 parameters):
  Memory estimate: 6.80 GB
  Chunk size: 1,000,000
  Number of chunks: 50
  Strategy: Chunked processing

Extremely large dataset (100,000,000 points, 3 parameters):
  Memory estimate: 13.60 GB
  Chunk size: 1,000,000
  Number of chunks: 100
  Strategy: Chunked processing


## 1.5. Advanced Memory Configuration and Algorithm Selection

NLSQ now provides sophisticated configuration management and automatic algorithm selection for optimal performance with large datasets.

In [3]:
def demo_advanced_configuration():
    """Demonstrate advanced configuration and algorithm selection."""
    print("=" * 60)
    print("ADVANCED CONFIGURATION & ALGORITHM SELECTION DEMO")
    print("=" * 60)

    # Current memory configuration
    current_config = get_memory_config()
    print("Current memory configuration:")
    print(f"  Memory limit: {current_config.memory_limit_gb} GB")
    print(
        f"  Mixed precision fallback: {current_config.enable_mixed_precision_fallback}"
    )

    # Automatically configure for large datasets
    print("\nConfiguring for large dataset processing...")
    configure_for_large_datasets(
        memory_limit_gb=8.0, enable_sampling=True, enable_chunking=True
    )

    # Show updated configuration
    new_config = get_memory_config()
    print(f"Updated memory limit: {new_config.memory_limit_gb} GB")

    # Generate test dataset for algorithm selection
    print("\n=== Algorithm Selection Demo ===")
    np.random.seed(42)

    # Test different model complexities
    test_cases = [
        ("Simple exponential", exponential_decay, 3, [5.0, 1.2, 0.5]),
        ("Polynomial", polynomial_model, 4, [0.1, -0.5, 2.0, 1.0]),
        ("Complex multi-param", complex_model, 6, [3.0, 0.8, 1.5, 2.0, 0.1, 0.2]),
    ]

    for model_name, model_func, n_params, true_params in test_cases:
        print(f"\n{model_name} ({n_params} parameters):")

        # Generate sample data
        n_sample = 10000  # Smaller sample for algorithm analysis
        x_sample = np.linspace(0, 5, n_sample)
        y_sample = model_func(x_sample, *true_params) + np.random.normal(
            0, 0.05, n_sample
        )

        # Get algorithm recommendation
        try:
            recommendations = auto_select_algorithm(model_func, x_sample, y_sample)

            print(f"  Recommended algorithm: {recommendations['algorithm']}")
            print(f"  Recommended tolerance: {recommendations['ftol']}")
            print(
                f"  Problem complexity: {recommendations.get('complexity', 'Unknown')}"
            )

            # Estimate memory for full dataset
            large_n = 1_000_000  # 1M points
            stats = estimate_memory_requirements(large_n, n_params)
            print(f"  Memory for 1M points: {stats.total_memory_estimate_gb:.3f} GB")
            print(
                f"  Chunking strategy: {'Required' if stats.n_chunks > 1 else 'Not needed'}"
            )
        except Exception as e:
            print(f"  Algorithm selection failed: {e}")
            print(f"  Using default settings for {model_name}")


# Run the demo
demo_advanced_configuration()

ADVANCED CONFIGURATION & ALGORITHM SELECTION DEMO
Current memory configuration:
  Memory limit: 8.0 GB
  Mixed precision fallback: True

Configuring for large dataset processing...
Updated memory limit: 8.0 GB

=== Algorithm Selection Demo ===

Simple exponential (3 parameters):


  Recommended algorithm: trf
  Recommended tolerance: 1e-08
  Problem complexity: Unknown
  Memory for 1M points: 0.136 GB
  Chunking strategy: Not needed

Polynomial (4 parameters):
  Recommended algorithm: trf
  Recommended tolerance: 1e-08
  Problem complexity: Unknown
  Memory for 1M points: 0.158 GB
  Chunking strategy: Not needed

Complex multi-param (6 parameters):
  Recommended algorithm: trf
  Recommended tolerance: 1e-08
  Problem complexity: Unknown
  Memory for 1M points: 0.203 GB
  Chunking strategy: Not needed


## 2. Basic Large Dataset Fitting

Let's demonstrate fitting a 1 million point dataset using the convenience function `fit_large_dataset`.

In [4]:
def demo_basic_large_dataset_fitting():
    """Demonstrate basic large dataset fitting."""
    print("\n" + "=" * 60)
    print("BASIC LARGE DATASET FITTING DEMO")
    print("=" * 60)

    # Generate synthetic large dataset (1M points)
    print("Generating 1M point exponential decay dataset...")
    np.random.seed(42)
    n_points = 1_000_000
    x_data = np.linspace(0, 5, n_points, dtype=np.float64)
    true_params = [5.0, 1.2, 0.5]
    noise_level = 0.05

    y_true = true_params[0] * np.exp(-true_params[1] * x_data) + true_params[2]
    y_data = y_true + np.random.normal(0, noise_level, n_points)

    print(f"Dataset: {n_points:,} points")
    print(
        f"True parameters: a={true_params[0]}, b={true_params[1]}, c={true_params[2]}"
    )

    # Fit using convenience function
    print("\nFitting with automatic memory management...")
    start_time = time.time()

    result = fit_large_dataset(
        exponential_decay,
        x_data,
        y_data,
        p0=[4.0, 1.0, 0.4],
        memory_limit_gb=2.0,  # 2GB limit
        show_progress=True,
    )

    fit_time = time.time() - start_time

    if result.success:
        fitted_params = np.array(result.popt)
        errors = np.abs(fitted_params - np.array(true_params))
        rel_errors = errors / np.array(true_params) * 100

        print(f"\n✅ Fit completed in {fit_time:.2f} seconds")
        print(
            f"Fitted parameters: [{fitted_params[0]:.3f}, {fitted_params[1]:.3f}, {fitted_params[2]:.3f}]"
        )
        print(f"Absolute errors: [{errors[0]:.4f}, {errors[1]:.4f}, {errors[2]:.4f}]")
        print(
            f"Relative errors: [{rel_errors[0]:.2f}%, {rel_errors[1]:.2f}%, {rel_errors[2]:.2f}%]"
        )
    else:
        print(f"❌ Fit failed: {result.message}")


# Run the demo
demo_basic_large_dataset_fitting()

INFO:nlsq.nlsq.large_dataset:Dataset analysis for 1,000,000 points, 3 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 146.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.14 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 1,000,000


INFO:nlsq.nlsq.large_dataset:  Number of chunks: 1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.769576s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=None | final_cost=1.250468e+03 | time=0.770s | final_gradient_norm=5.634339999005533e-07


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 1.026983s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 1.0269832080011838, 'final_cost': 2500.93619980661, 'covariance_warning': False}



✅ Fit completed in 1.11 seconds
Fitted parameters: [5.000, 1.200, 0.500]
Absolute errors: [0.0002, 0.0000, 0.0001]
Relative errors: [0.00%, 0.00%, 0.03%]


## 3.5. Context Managers and Temporary Configuration

NLSQ provides context managers for temporary configuration changes, allowing you to optimize settings for specific operations without affecting global state.

In [5]:
def demo_context_managers():
    """Demonstrate context managers for temporary configuration."""
    print("\n" + "=" * 60)
    print("CONTEXT MANAGERS DEMO")
    print("=" * 60)

    # Show current configuration
    original_mem_config = get_memory_config()
    print(f"Original memory limit: {original_mem_config.memory_limit_gb} GB")

    # Generate test data
    np.random.seed(555)
    n_points = 500_000
    x_data = np.linspace(0, 5, n_points)
    y_data = exponential_decay(x_data, 4.0, 1.5, 0.3) + np.random.normal(
        0, 0.05, n_points
    )

    print(f"Test dataset: {n_points:,} points")

    # Test 1: Memory context for memory-constrained fitting
    print("\n--- Test 1: Memory-constrained fitting ---")
    constrained_config = MemoryConfig(
        memory_limit_gb=0.5,  # Very low limit
        enable_mixed_precision_fallback=True,
    )

    with memory_context(constrained_config):
        temp_config = get_memory_config()
        print(f"Inside context memory limit: {temp_config.memory_limit_gb} GB")
        print(f"Mixed precision enabled: {temp_config.enable_mixed_precision_fallback}")

        start_time = time.time()
        result1 = fit_large_dataset(
            exponential_decay, x_data, y_data, p0=[3.5, 1.3, 0.25], show_progress=False
        )
        time1 = time.time() - start_time

        if result1.success:
            print(f"✅ Constrained fit completed: {time1:.3f}s")
            print(f"   Parameters: {result1.popt}")
        else:
            print(f"❌ Constrained fit failed: {result1.message}")

    # Check that configuration is restored
    restored_config = get_memory_config()
    print(f"After context memory limit: {restored_config.memory_limit_gb} GB")

    # Test 2: Large dataset context for optimized processing
    print("\n--- Test 2: Large dataset optimization ---")
    ld_config = LargeDatasetConfig(
        enable_sampling=False,  # Force chunking instead of sampling
        max_sampled_size=100_000,
        sampling_threshold=1_000_000,
    )

    with large_dataset_context(ld_config):
        print("Inside large dataset context - chunking optimized")

        start_time = time.time()
        result2 = fit_large_dataset(
            exponential_decay, x_data, y_data, p0=[3.5, 1.3, 0.25], show_progress=False
        )
        time2 = time.time() - start_time

        if result2.success:
            print(f"✅ Optimized fit completed: {time2:.3f}s")
            print(f"   Parameters: {result2.popt}")
        else:
            print(f"❌ Optimized fit failed: {result2.message}")

    # Test 3: Combined context for specific algorithm
    print("\n--- Test 3: Algorithm-specific optimization ---")

    # Get algorithm recommendation first
    sample_size = 5000
    x_sample = x_data[:sample_size]
    y_sample = y_data[:sample_size]
    recommendations = auto_select_algorithm(exponential_decay, x_sample, y_sample)

    print(f"Recommended algorithm: {recommendations['algorithm']}")
    print(f"Recommended tolerance: {recommendations['ftol']}")

    # Use CurveFit with recommended settings
    optimized_config = MemoryConfig(
        memory_limit_gb=2.0, enable_mixed_precision_fallback=True
    )

    with memory_context(optimized_config):
        start_time = time.time()

        # Use the regular CurveFit for comparison
        cf = CurveFit(use_dynamic_sizing=True)
        popt3, pcov3 = cf.curve_fit(
            exponential_decay,
            x_data,
            y_data,
            p0=[3.5, 1.3, 0.25],
            ftol=recommendations.get("ftol", 1e-8),
        )
        time3 = time.time() - start_time

        print(f"✅ Algorithm-optimized fit completed: {time3:.3f}s")
        print(f"   Parameters: {popt3}")
        print(f"   Parameter uncertainties: {np.sqrt(np.diag(pcov3))}")

    # Compare all approaches
    if result1.success and result2.success:
        print("\n=== Performance Comparison ===")
        print(f"Constrained memory: {time1:.3f}s")
        print(f"Chunking optimized: {time2:.3f}s")
        print(f"Algorithm optimized: {time3:.3f}s")

        # Calculate accuracy
        true_params = [4.0, 1.5, 0.3]
        errors1 = np.abs(result1.popt - true_params)
        errors2 = np.abs(result2.popt - true_params)
        errors3 = np.abs(popt3 - true_params)

        print("\nAccuracy comparison (absolute errors):")
        print(f"Constrained: {errors1}")
        print(f"Chunking:    {errors2}")
        print(f"Algorithm:   {errors3}")

    print("\n✓ Context managers allow flexible, temporary configuration changes!")


# Run the demo
demo_context_managers()

INFO:nlsq.nlsq.large_dataset:Dataset analysis for 500,000 points, 3 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 146.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.07 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 500,000


INFO:nlsq.nlsq.large_dataset:  Number of chunks: 1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.420963s


INFO:nlsq.least_squares:Convergence: reason=Both `ftol` and `xtol` termination conditions are satisfied. | iterations=None | final_cost=6.277286e+02 | time=0.421s | final_gradient_norm=1.6682892400865512e-08


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.653990s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.6539900009993289, 'final_cost': 1255.4571180397163, 'covariance_warning': False}


✅ Algorithm-optimized fit completed: 0.722s
   Parameters: [4.00017172 1.49998995 0.29995423]
   Parameter uncertainties: [0.00038815 0.00025672 0.00010319]

=== Performance Comparison ===
Constrained memory: 0.743s
Chunking optimized: 0.695s
Algorithm optimized: 0.722s

Accuracy comparison (absolute errors):
Constrained: [1.71720945e-04 1.00451087e-05 4.57656340e-05]
Chunking:    [1.71720945e-04 1.00451087e-05 4.57656340e-05]
Algorithm:   [1.71720945e-04 1.00451087e-05 4.57656340e-05]

✓ Context managers allow flexible, temporary configuration changes!


## 3. Chunked Processing Demo

For datasets that don't fit in memory, NLSQ automatically chunks the data and processes it in batches.

In [6]:
def demo_chunked_processing():
    """Demonstrate chunked processing with progress reporting."""
    print("\n" + "=" * 60)
    print("CHUNKED PROCESSING DEMO")
    print("=" * 60)

    # Generate a dataset that will require chunking
    print("Generating 2M point polynomial dataset...")
    np.random.seed(123)
    n_points = 2_000_000
    x_data = np.linspace(-2, 2, n_points, dtype=np.float64)
    true_params = [0.5, -1.2, 2.0, 1.5]
    noise_level = 0.1

    y_true = (
        true_params[0] * x_data**3
        + true_params[1] * x_data**2
        + true_params[2] * x_data
        + true_params[3]
    )
    y_data = y_true + np.random.normal(0, noise_level, n_points)

    print(f"Dataset: {n_points:,} points")
    print(f"True parameters: {true_params}")

    # Create fitter with limited memory to force chunking
    fitter = LargeDatasetFitter(memory_limit_gb=0.5)  # Small limit to force chunking

    # Get processing recommendations
    recs = fitter.get_memory_recommendations(n_points, 4)
    print(f"\nProcessing strategy: {recs['processing_strategy']}")
    print(f"Chunk size: {recs['recommendations']['chunk_size']:,}")
    print(f"Number of chunks: {recs['recommendations']['n_chunks']}")
    print(
        f"Memory estimate: {recs['recommendations']['total_memory_estimate_gb']:.2f} GB"
    )

    # Fit with progress reporting
    print("\nFitting with chunked processing...")
    start_time = time.time()

    result = fitter.fit_with_progress(
        polynomial_model, x_data, y_data, p0=[0.4, -1.0, 1.8, 1.2]
    )

    fit_time = time.time() - start_time

    if result.success:
        fitted_params = np.array(result.popt)
        errors = np.abs(fitted_params - np.array(true_params))
        rel_errors = errors / np.abs(np.array(true_params)) * 100

        print(f"\n✅ Chunked fit completed in {fit_time:.2f} seconds")
        if hasattr(result, "n_chunks"):
            print(
                f"Used {result.n_chunks} chunks with {result.success_rate:.1%} success rate"
            )
        print(f"Fitted parameters: {fitted_params}")
        print(f"Absolute errors: {errors}")
        print(f"Relative errors: {rel_errors}%")
    else:
        print(f"❌ Chunked fit failed: {result.message}")


# Run the demo
demo_chunked_processing()

INFO:nlsq.nlsq.large_dataset:Dataset analysis for 2,000,000 points, 4 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 170.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.32 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 1,000,000


INFO:nlsq.nlsq.large_dataset:  Number of chunks: 2


...

PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.037874s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.03787448100047186, 'final_cost': 10009.661564309994, 'covariance_warning': False}


INFO:nlsq.nlsq.large_dataset:Progress: 2/2 chunks (100.0%) - ETA: 0.0s


INFO:nlsq.nlsq.large_dataset:Chunked fit completed with 100.0% success rate



✅ Chunked fit completed in 1.11 seconds
Used 2 chunks with 100.0% success rate
Fitted parameters: [ 0.49954434 -1.1991878   2.00000184  1.49975205]
Absolute errors: [4.55663613e-04 8.12197789e-04 1.84432241e-06 2.47951909e-04]
Relative errors: [9.11327226e-02 6.76831491e-02 9.22161203e-05 1.65301273e-02]%


## 4. Sampling Strategy for Extremely Large Datasets

For datasets with 100M+ points, sampling strategies can be more efficient than processing all data.

In [7]:
def demo_sampling_strategy():
    """Demonstrate sampling for extremely large datasets."""
    print("\n" + "=" * 60)
    print("SAMPLING STRATEGY DEMO")
    print("=" * 60)

    # Simulate a very large dataset scenario
    print("Simulating extremely large dataset (100M points)...")
    n_points_full = 100_000_000  # 100M points
    true_params = [3.0, 0.8, 0.2]

    # For demo purposes, generate a smaller representative sample
    # In practice, you would have this data already or stream it
    np.random.seed(456)
    n_sample = 1_000_000  # 1M sample for demo
    x_sample = np.sort(np.random.uniform(0, 5, n_sample))
    y_sample = (
        true_params[0] * np.exp(-true_params[1] * x_sample)
        + true_params[2]
        + np.random.normal(0, 0.05, n_sample)
    )

    print(f"Full dataset size: {n_points_full:,} points (simulated)")
    print(f"Demo sample size: {n_sample:,} points")
    print(f"True parameters: {true_params}")

    # Check memory requirements for full dataset
    stats = estimate_memory_requirements(n_points_full, 3)
    print(f"\nFull dataset memory estimate: {stats.total_memory_estimate_gb:.2f} GB")
    print(f"Sampling recommended: {stats.requires_sampling}")

    # Create fitter with sampling enabled
    config = LDMemoryConfig(memory_limit_gb=4.0, enable_sampling=True)
    fitter = LargeDatasetFitter(config=config)

    print("\nFitting with sampling strategy...")
    start_time = time.time()

    # For demo, use our sample as if it were the full dataset
    result = fitter.fit(exponential_decay, x_sample, y_sample, p0=[2.5, 1.0, 0.1])

    fit_time = time.time() - start_time

    if result.success:
        fitted_params = np.array(result.popt)
        errors = np.abs(fitted_params - np.array(true_params))
        rel_errors = errors / np.array(true_params) * 100

        print(f"\n✅ Sampling fit completed in {fit_time:.2f} seconds")
        print(f"Fitted parameters: {fitted_params}")
        print(f"Absolute errors: {errors}")
        print(f"Relative errors: {rel_errors}%")

        if hasattr(result, "was_sampled") and result.was_sampled:
            print(
                f"Used sampling: {result.sample_size:,} points from {result.original_size:,}"
            )
    else:
        print(f"❌ Sampling fit failed: {result.message}")


# Run the demo
demo_sampling_strategy()

INFO:nlsq.nlsq.large_dataset:Dataset analysis for 1,000,000 points, 3 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 146.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.14 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 1,000,000


INFO:nlsq.nlsq.large_dataset:  Number of chunks: 1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.461450s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=None | final_cost=1.248005e+03 | time=0.461s | final_gradient_norm=2.131660789217449e-06


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.678390s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.678389832000903, 'final_cost': 2496.009988407105, 'covariance_warning': False}



✅ Sampling fit completed in 0.76 seconds
Fitted parameters: [2.99987792 0.79974645 0.19984664]
Absolute errors: [0.00012208 0.00025355 0.00015336]
Relative errors: [0.00406948 0.03169369 0.07668245]%


## 5. curve_fit_large Convenience Function

The `curve_fit_large` function provides automatic detection and handling of large datasets, making it easy to switch between standard and large dataset processing.

In [8]:
def demo_curve_fit_large():
    """Demonstrate the curve_fit_large convenience function."""
    print("\n" + "=" * 60)
    print("CURVE_FIT_LARGE CONVENIENCE FUNCTION DEMO")
    print("=" * 60)

    # Generate test dataset
    print("Generating 3M point dataset for curve_fit_large demo...")
    np.random.seed(789)
    n_points = 3_000_000
    x_data = np.linspace(0, 10, n_points, dtype=np.float64)

    true_params = [5.0, 5.0, 1.5, 0.5]
    y_true = gaussian(x_data, *true_params)
    y_data = y_true + np.random.normal(0, 0.1, n_points)

    print(f"Dataset: {n_points:,} points")
    print(
        f"True parameters: a={true_params[0]:.2f}, mu={true_params[1]:.2f}, sigma={true_params[2]:.2f}, offset={true_params[3]:.2f}"
    )

    # Use curve_fit_large - automatic large dataset handling
    print("\nUsing curve_fit_large with automatic optimization...")
    start_time = time.time()

    popt, pcov = curve_fit_large(
        gaussian,
        x_data,
        y_data,
        p0=[4.5, 4.8, 1.3, 0.4],
        memory_limit_gb=1.0,  # Force chunking with low memory limit
        show_progress=True,
        auto_size_detection=True,  # Automatically detect large dataset
    )

    fit_time = time.time() - start_time

    errors = np.abs(popt - np.array(true_params))
    rel_errors = errors / np.array(true_params) * 100

    print(f"\n✅ curve_fit_large completed in {fit_time:.2f} seconds")
    print(f"Fitted parameters: {popt}")
    print(f"Absolute errors: {errors}")
    print(f"Relative errors: {rel_errors}%")

    # Show parameter uncertainties from covariance matrix
    param_std = np.sqrt(np.diag(pcov))
    print(f"Parameter uncertainties (std): {param_std}")


# Run the demo
demo_curve_fit_large()


CURVE_FIT_LARGE CONVENIENCE FUNCTION DEMO
Generating 3M point dataset for curve_fit_large demo...


INFO:nlsq.nlsq.large_dataset:Dataset analysis for 3,000,000 points, 4 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 170.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.47 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 300,000


...

PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.049545s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.04954509700110066, 'final_cost': 3012.2084416558946, 'covariance_warning': False}


INFO:nlsq.nlsq.large_dataset:Progress: 10/10 chunks (100.0%) - ETA: 0.0s


INFO:nlsq.nlsq.large_dataset:Chunked fit completed with 100.0% success rate



✅ curve_fit_large completed in 2.81 seconds
Fitted parameters: [22.91920679  3.25963575  1.8053707   0.49727729]
Absolute errors: [1.79192068e+01 1.74036425e+00 3.05370696e-01 2.72270809e-03]
Relative errors: [358.38413589  34.80728503  20.35804638   0.54454162]%
Parameter uncertainties (std): [6.15079165 0.810103   0.16448884 0.13548504]


## 6. Performance Comparison

Let's compare different approaches for various dataset sizes.

In [9]:
def compare_approaches():
    """Compare different fitting approaches."""
    print("\n" + "=" * 60)
    print("PERFORMANCE COMPARISON")
    print("=" * 60)

    # Test different dataset sizes
    sizes = [10_000, 100_000, 500_000]

    print(f"\n{'Size':>10} {'Time (s)':>12} {'Memory (GB)':>12} {'Strategy':>20}")
    print("-" * 55)

    for n in sizes:
        # Generate data
        np.random.seed(42)
        x = np.linspace(0, 10, n)
        y = 2.0 * np.exp(-0.5 * x) + 0.3 + np.random.normal(0, 0.05, n)

        # Get memory estimate
        stats = estimate_memory_requirements(n, 3)

        # Determine strategy
        if stats.n_chunks == 1:
            strategy = "Single chunk"
        elif stats.requires_sampling:
            strategy = "Sampling"
        else:
            strategy = f"Chunked ({stats.n_chunks} chunks)"

        # Time the fit
        start = time.time()
        result = fit_large_dataset(
            exponential_decay,
            x,
            y,
            p0=[2.5, 0.6, 0.2],
            memory_limit_gb=0.5,  # Small limit to test chunking
            show_progress=False,
        )
        elapsed = time.time() - start

        print(
            f"{n:10,} {elapsed:12.3f} {stats.total_memory_estimate_gb:12.3f} {strategy:>20}"
        )


# Run comparison
compare_approaches()

INFO:nlsq.nlsq.large_dataset:Dataset analysis for 10,000 points, 3 parameters:


INFO:nlsq.nlsq.large_dataset:  Estimated memory per point: 146.0 bytes


INFO:nlsq.nlsq.large_dataset:  Total memory estimate: 0.00 GB


INFO:nlsq.nlsq.large_dataset:  Recommended chunk size: 10,000


INFO:nlsq.nlsq.large_dataset:  Number of chunks: 1


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.410136s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=None | final_cost=6.255845e+02 | time=0.410s | final_gradient_norm=3.558978001194646e-08


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.651783s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.6517829780004831, 'final_cost': 1251.1690277572839, 'covariance_warning': False}


   500,000        0.716        0.068         Single chunk


## Summary and Key Takeaways

NLSQ provides comprehensive support for large dataset fitting with recent improvements:

1. **Automatic Memory Management**: NLSQ automatically detects available memory and chooses the best strategy
2. **Improved Chunking Algorithm**: Advanced exponential moving average approach achieves <1% error for well-conditioned problems
3. **JAX Tracing Compatibility**: Supports functions with up to 15+ parameters without TracerArrayConversionError
4. **curve_fit_large Function**: Automatic dataset size detection and intelligent processing strategy selection
5. **Sampling Strategies**: For extremely large datasets (>100M points), intelligent sampling can provide accurate results
6. **Progress Reporting**: Long-running fits provide progress updates
7. **Memory Estimation**: Predict memory requirements before fitting

### Best Practices:

- Use `curve_fit_large()` for automatic handling of both small and large datasets
- Use `estimate_memory_requirements()` to understand dataset requirements
- Use `fit_large_dataset()` when you need explicit control over large dataset processing
- Set appropriate `memory_limit_gb` based on your system
- Enable sampling for datasets >100M points
- Use progress reporting for long-running fits

### Recent Improvements (v810dc5c):

- **Fixed JAX tracing issues** for functions with many parameters
- **Enhanced chunking algorithm** with adaptive learning rates and convergence monitoring
- **Ensured return type consistency** across all code paths
- **Added comprehensive test coverage** for large dataset functionality

In [10]:
# Print final summary
print("\n" + "=" * 60)
print("DEMO COMPLETED")
print("=" * 60)
print("\nKey takeaways:")
print("• NLSQ automatically handles memory management for large datasets")
print("• Chunked processing works for datasets that don't fit in memory")
print("• curve_fit_large provides automatic dataset size detection")
print("• Improved chunking algorithm achieves <1% error for well-conditioned problems")
print("• Sampling strategies can handle extremely large datasets efficiently")
print("• Progress reporting helps track long-running fits")
print("• Memory estimation helps plan processing strategies")


DEMO COMPLETED

Key takeaways:
• NLSQ automatically handles memory management for large datasets
• Chunked processing works for datasets that don't fit in memory
• curve_fit_large provides automatic dataset size detection
• Improved chunking algorithm achieves <1% error for well-conditioned problems
• Sampling strategies can handle extremely large datasets efficiently
• Progress reporting helps track long-running fits
• Memory estimation helps plan processing strategies
